In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import confusion_matrix , classification_report 
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score

from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

from keras.models import model_from_json
import cv2, os
from keras.layers import Flatten
from keras.utils.vis_utils import plot_model

from keras.layers import Conv2D,MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D

In [3]:
train_dir = r"dataset/Binarydataset/train_model" 
test_dir = r"dataset/Binarydataset/test_model"

SEED = 12
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 64
EPOCHS = 500
#FINE_TUNING_EPOCHS = 30
LR = 0.001
NUM_CLASSES = 8
EARLY_STOPPING_CRITERIA=3
CLASS_LABELS  = ['Amusement', 'Anger', 'Awe', 'Contentment', 'Disgust', 'Excitement', 'Fear', 'Sadness']
CLASS_LABELS_EMOJIS = ["🥳", "😡", "😯", "😌", "🤢" ,"🤩", "😱" , "😔" ]

In [4]:
preprocess_fun = tf.keras.applications.densenet.preprocess_input

train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.05,
                                   rotation_range= 10,
                                   rescale = 1./255,
                                   validation_split = 0,
                                   preprocessing_function=preprocess_fun
                                  )
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0,
                                  preprocessing_function=preprocess_fun)

train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = True , 
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    subset = "training",
                                                    seed = 12
                                                   )

test_generator = test_datagen.flow_from_directory(directory = test_dir,
                                                   target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = False , 
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    seed = 12
                                                  )

Found 18009 images belonging to 2 classes.
Found 3277 images belonging to 2 classes.


In [5]:
def classifier(inputs):
    x = Conv2D(96,(11,11), strides = (4,4) , input_shape = (224,224,3), activation = 'relu')(inputs)
    x = MaxPooling2D(pool_size=(3,3), strides= (2,2))(x)
    x = tf.keras.layers.Normalization()(x)
    x = Conv2D(256,(5,5), strides = (2,2), activation = 'relu')(x)
    x = MaxPooling2D(pool_size=(3,3), strides= (2,2))(x)
    x = tf.keras.layers.Normalization()(x)
    x = Conv2D(96,(3,3), strides = (1,1), activation = 'relu')(x)
    x = MaxPooling2D(pool_size=(3,3), strides= (2,2))(x)
    x = tf.keras.layers.Normalization()(x)
    x = Flatten()(x)
    x = tf.keras.layers.Dense(4096, activation='relu')(x)
    x = tf.keras.layers.Dense(4096, activation='relu')(x)
    x = tf.keras.layers.Dense(2, activation = 'softmax')(x)
    
    return x

def final_model(inputs):
    classification_output = classifier(inputs)
    
    return classification_output

def define_compile_model():
    
    inputs = tf.keras.layers.Input(shape=(224,224,3))
    classification_output = final_model(inputs) 
    model = tf.keras.Model(inputs= inputs, outputs= classification_output)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), 
                loss='categorical_crossentropy',
                metrics = ['accuracy'])
  
    return model

In [7]:
model = define_compile_model()

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 26, 26, 96)       0         
 2D)                                                             
                                                                 
 normalization_3 (Normalizat  (None, 26, 26, 96)       193       
 ion)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 256)       614656    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 5, 5, 256)        0   

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('Binary-{epoch:03d}-{accuracy:03f}-{val_accuracy:03f}.h5', verbose=1, 
                             monitor='val_accuracy',save_best_only=True, mode='auto') 
# earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='auto')

In [7]:
history = model.fit(train_generator, batch_size = BATCH_SIZE,
                   epochs= 100, validation_data= test_generator,
                   validation_batch_size = BATCH_SIZE, callbacks= [checkpoint]
                   )

history = pd.DataFrame(history.history)

Epoch 1/100
282/282 [==============================] - ETA: 0s - loss: 0.5811 - accuracy: 0.7074
Epoch 1: val_accuracy improved from -inf to 0.73055, saving model to Binary-001-0.707369-0.730546.h5
282/282 [==============================] - 372s 1s/step - loss: 0.5811 - accuracy: 0.7074 - val_loss: 0.5355 - val_accuracy: 0.7305
Epoch 2/100
282/282 [==============================] - ETA: 0s - loss: 0.5388 - accuracy: 0.7309
Epoch 2: val_accuracy improved from 0.73055 to 0.74062, saving model to Binary-002-0.730912-0.740616.h5
282/282 [==============================] - 262s 931ms/step - loss: 0.5388 - accuracy: 0.7309 - val_loss: 0.5194 - val_accuracy: 0.7406
Epoch 3/100
282/282 [==============================] - ETA: 0s - loss: 0.5298 - accuracy: 0.7370
Epoch 3: val_accuracy improved from 0.74062 to 0.74428, saving model to Binary-003-0.737020-0.744278.h5
282/282 [==============================] - 271s 960ms/step - loss: 0.5298 - accuracy: 0.7370 - val_loss: 0.5158 - val_accuracy: 0.744

In [8]:
#saving the  model to be used later
fer_json = model.to_json()
with open("Binary.json", "w") as json_file:
    json_file.write(fer_json)
#model.save_weights("VGG16_512.h5") #make sure to rename this file after each 100 epochs
print("Saved model somewhere")

Saved model somewhere
